In [ ]:
"""Preprocesses data for training.

Preferrably, run using Jupyter Notebook.
"""
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
# Read raw data from the excel file.
raw_data = pd.read_excel(
    "data/ouse93-96-raw-data.xlsx",
    index_col=0,
    header=1,
    usecols='A:I'
)

In [ ]:
# Convert columns to float values. Turn non-numeric values into np.nan.
for column in raw_data.columns[1:]:
    raw_data[column] = pd.to_numeric(raw_data[column], errors='coerce')

In [ ]:
# Convert negative values into np.nan.
raw_data = raw_data.mask(raw_data < 0, np.nan)

In [ ]:
# Print all rows with NAN values.
raw_data[raw_data.isna().any(axis=1)]

In [ ]:
# Calculate mean and standard deviation for each numeric column.
standard_deviations = raw_data.std()
means = raw_data.mean()

In [ ]:
# Plot river flow columns.
raw_data[['Crakehill', 'Skip Bridge', 'Westwick', 'Skelton']].plot(xlabel="Date", ylabel="Mean Daily Flow [Cumecs]")
plt.savefig('figures/river-flow.png') 

In [ ]:
# Find the values that lies the furthest from the mean from Mean Daily Flow columns.
skelton_max = raw_data['Skelton'].max()
print(raw_data['Skelton'].sort_values(ascending=False).head())
print((skelton_max - means['Skelton']) / standard_deviations['Skelton'])


In [ ]:
# Plot river flow columns.
raw_data[['Arkengarthdale', 'East Cowton', 'Malham Tarn', 'Snaizeholme']].plot(xlabel="Date", ylabel="Daily Rainfall Total [mm]")
plt.savefig('figures/rainfall.png')

print(raw_data['Arkengarthdale'].sort_values(ascending=False).head(1))
print(raw_data['East Cowton'].sort_values(ascending=False).head(1))
print(raw_data['Malham Tarn'].sort_values(ascending=False).head(1))

In [ ]:
# Interpolate outliers.
for column in raw_data[['Arkengarthdale', 'East Cowton', 'Malham Tarn']].columns:
    column_values = raw_data[column]
    column_max_id = column_values.idxmax()
    column_values.loc[column_max_id] = np.nan

    """
    column_values = column_values.mask(
        column_values > means[column] + 3 * standard_deviations[column],
        np.nan
    )
    
    # If this is a river flow column, check the lower bound as well.
    if column in ['Crakehill', 'Skip Bridge', 'Westwick', 'Skelton']:
        column_values = column_values.mask(
            column_values < means[column] - 3 * standard_deviations[column],
            np.nan
        )
    raw_data[column] = column_values.interpolate(method="linear") 
    """

In [ ]:
# Imputate spurious data.
raw_data = raw_data.interpolate(method="linear")

In [ ]:
# Print all rows with NAN values.
raw_data[raw_data.isna().any(axis=1)]

In [ ]:
# Print values interpolated for outliers.
print(raw_data.loc["1995-02-11", "Arkengarthdale"])
print(raw_data.loc["1995-02-28", "East Cowton"])
print(raw_data.loc["1996-01-10", "Malham Tarn"])


In [ ]:
# Print all rows with NAN values.
raw_data[raw_data.isna().any(axis=1)]

In [ ]:
# Add a column for Mean Daily Flow in Skelton one day before. 
raw_data.insert(0, 'Skelton T-1', raw_data['Skelton'])

In [ ]:
# Make predictand column the rightmost column.
columns = raw_data.columns
skelton_column = raw_data.pop("Skelton")
raw_data.insert(len(columns) - 1, skelton_column.name, skelton_column)

In [ ]:
# Explore moving average

#weights = np.array([0.5, 0.25, 0.25])
#sum_weights = np.sum(weights)

raw_data.insert(0, "Arkengarthdale MA", raw_data['Arkengarthdale'].rolling(3).mean())
raw_data.insert(0, "East Cowton MA", raw_data['East Cowton'].rolling(3).mean())
raw_data.insert(0, "Malham Tarn MA", raw_data['Malham Tarn'].rolling(3).mean())
raw_data.insert(0, "Snaizeholme MA", raw_data['Snaizeholme'].rolling(3).mean())
raw_data["Snaizeholme MA"]["1993-01-01"]

In [ ]:
# Explore the correlations between potential predictors and the predictand.

print(raw_data.corr()["Skelton"].sort_values(ascending=False))
# Lag Mean Daily Flow potential predictor columns by one day.
raw_data['Skelton T-1'] = raw_data['Skelton T-1'].shift(periods=1, freq="D")
raw_data['Crakehill'] = raw_data['Crakehill'].shift(periods=1, freq="D")
raw_data['Skip Bridge'] = raw_data['Skip Bridge'].shift(periods=1, freq="D")
raw_data['Westwick'] = raw_data['Westwick'].shift(periods=1, freq="D")

# Lag Rainfall columns by one day.
raw_data['Arkengarthdale MA'] = raw_data['Arkengarthdale MA'].shift(periods=1, freq="D")
raw_data['East Cowton MA'] = raw_data['East Cowton MA'].shift(periods=1, freq="D")
raw_data['Malham Tarn MA'] = raw_data['Malham Tarn MA'].shift(periods=1, freq="D")
raw_data['Snaizeholme MA'] = raw_data['Snaizeholme MA'].shift(periods=1, freq="D")
raw_data['Arkengarthdale'] = raw_data['Arkengarthdale'].shift(periods=1, freq="D")
raw_data['East Cowton'] = raw_data['East Cowton'].shift(periods=1, freq="D")
raw_data['Malham Tarn'] = raw_data['Malham Tarn'].shift(periods=1, freq="D")
raw_data['Snaizeholme'] = raw_data['Snaizeholme'].shift(periods=1, freq="D")

correlations = raw_data.corr()
correlations["Skelton"].sort_values(ascending=False)

In [ ]:
# Plot correlations
correlations_to_plot = correlations.drop(index=["Skelton", "Skelton T-1", "Crakehill", "Skip Bridge", "Westwick"])["Skelton"].sort_values(ascending=False).plot(xlabel="Potential Rainfall Predictors", ylabel="Correlation with the predictand", kind="bar")
plt.savefig('figures/rainfall-correlations.png') 

In [ ]:
# Drop rows containing missing values after lagging.
raw_data = raw_data.dropna()
raw_data

In [ ]:
# Drop non-moving-average rainfall columns.
raw_data = raw_data.drop(columns=['Arkengarthdale', 'East Cowton', 'Malham Tarn', 'Snaizeholme'])

In [ ]:
# Plot correlation between lagged predictors and the predictand.

fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(15,15))

plt.subplots_adjust(wspace=0.2, hspace=0.5)

mean_daily_flow_columns = ["Skelton T-1", "Crakehill", "Skip Bridge", "Westwick"]
daily_rainfall_total_columns = ["Arkengarthdale MA", "East Cowton MA", "Malham Tarn MA", "Snaizeholme MA"]


for i in range(len(raw_data.loc[:, raw_data.columns != "Skelton"].columns)):
    column_name = raw_data.iloc[:, i].name
    x = raw_data.iloc[:, i].values.reshape(-1, 1)
    y = raw_data['Skelton'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(x, y)
    y_pred = linear_regressor.predict(x)
    axes[i // 2][i % 2].scatter(x, y, alpha=0.4)
    axes[i // 2][i % 2].plot(x, y_pred, color='red')
    # Decide on x-label.
    if column_name in mean_daily_flow_columns:
        axes[i // 2][ i % 2].set_xlabel(f"Mean Daily Flow in {column_name} [cumecs]")
    elif column_name in daily_rainfall_total_columns:
        axes[i // 2][ i % 2].set_xlabel(f"Daily Rainfall Total in {column_name} [mm]")
    axes[i // 2][ i % 2].set_xlabel('Mean Daily Flow in Skelton [cumecs]')
    axes[i // 2][ i % 2].set_title(f"{column_name} to Mean Daily Flow in Skelton")

plt.savefig('figures/predictors-predictand.png')
    


In [ ]:
# Split the data into into training-validation and test sets.
train_and_validation, test = train_test_split(raw_data, test_size=0.2, random_state=5)

# Calculate min and max values from training-validation set.
max_values = train_and_validation.max(axis=0)
min_values = train_and_validation.min(axis=0)
print(max_values)
print(min_values)

# Split training-validation set into training and validation sets.
train, validation = train_test_split(train_and_validation, test_size=0.25, random_state=6)
validation['Skelton'].head()

In [ ]:
def standardise(x: pd.Series, max_value: float, min_value: float):
    """Standardises data using minimum and maximum values.
    
    Args:
    x: A pandas.Series instance.
    max_value: A maximum value for the standardisation formula.
    min_value: A minimum value for the standardisation formula.
    
    Returns:
    pandas.Series.
    """
    return 0.8 * ((x - min_value) / (max_value - min_value)) + 0.1
    
# Standardise the data.
standardised_data_sets = []
for data_set in [train, validation, test]:
    standardised_columns = []
    for column in data_set.columns:
        standardised_column = data_set.loc[:, column].apply(standardise, args=(max_values[column], min_values[column]))
        standardised_columns.append(standardised_column)
    standardised_data_set = pd.concat(standardised_columns, axis=1)
    standardised_data_sets.append(standardised_data_set)
        
train_standardised = standardised_data_sets[0]
validation_standardised = standardised_data_sets[1]
test_standardised = standardised_data_sets[2]
train_standardised

In [ ]:
def destandardise(x: pd.Series, max_value: float, min_value: float):
    """Destandardises data using minimum and maximum values.
    
    Args:
    x: A pandas.Series instance of standardised data.
    max_value: A maximum value for the destandardisation formula.
    min_value: A minimum value for the destandardisation formula.
    
    Returns:
    pandas.Series.
    """
    return ((x - 0.1) * (max_value - min_value)) / 0.8 + min_value

In [ ]:
destandardised_data_sets = []
for data_set in standardised_data_sets:
    destandardised_columns = []
    for column in data_set.columns:
        destandardised_column = data_set.loc[:, column].apply(destandardise, args=(max_values[column], min_values[column]))
        destandardised_columns.append(destandardised_column)
    destandardised_data_set = pd.concat(destandardised_columns, axis=1)
    destandardised_data_sets.append(destandardised_data_set)
#destandardised_data_sets[0]
destandardised_data_sets[0].compare(train)
print(train.dtypes)
print(destandardised_data_sets[0].dtypes)
#train.to_csv("hello.csv")

In [ ]:
# Save min and max values
skelton_min_value = min_values['Skelton']
skelton_max_value = max_values['Skelton']


min_max_values = {
    "min": skelton_min_value,
    "max": skelton_max_value
}

with open("standardisation.json", "w") as f:
    json.dump(min_max_values, f)

In [ ]:
train_standardised.to_csv("data/training-set.csv")
validation_standardised.to_csv("data/validation-set.csv")
test_standardised.to_csv("data/test-set.csv")